# 书籍商品推荐系统

## 使用矩阵分解, 根据用户给书籍商品的评分数据, 做一个千人千面的个性化推荐系统
### 需要安装推荐系统库surprise, 使用如下命令安装: pip install scikit-surprise
## 需要自己补全带<span style="color:red">TODO</span>部分的代码

In [1]:
# 导入 nunpy 和 surprise 辅助库
import numpy as np
import surprise  

### Surprise库本身没有提供纯粹的矩阵分解的算法, 在这里我们自己实现了基于Alternating Least Squares的矩阵分解, 使用梯度下降法优化
### 矩阵分解类MatrixFactorization继承了surprise.AlgoBase, 方便我们使用surpise库提供的其它功能

In [2]:
class MatrixFactorization(surprise.AlgoBase):
    '''基于矩阵分解的推荐.'''
    
    def __init__(self, learning_rate, n_epochs, n_factors, lmd):
        
        self.lr = learning_rate  # 梯度下降法的学习率
        self.n_epochs = n_epochs  # 梯度下降法的迭代次数
        self.n_factors = n_factors  # 分解的矩阵的秩(rank)
        self.lmd = lmd # 防止过拟合的正则化的强度
        
    def fit(self, trainset):
        '''通过梯度下降法训练, 得到所有 u_i 和 p_j 的值'''
        
        print('Fitting data with SGD...')
        
        # 随机初始化 user 和 item 矩阵.
        u = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        p = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        
        # 梯度下降法
        for _ in range(self.n_epochs):
            for i, j, r_ij in trainset.all_ratings():
                err = r_ij - np.dot(u[i], p[j])
                TODO
                # 利用梯度调整 u_i 和 p_j
                # 注意: 修正 p_j 时, 安装严格定义, 我们应该使用 u_i 修正之前的值, 但是实际上差别微乎其微
        
        self.u, self.p = u, p
        self.trainset = trainset

    def estimate(self, i, j):
        '''预测 user i 对 item j 的评分.'''
        
        # 如果用户 i 和物品 j 是已知的值, 返回 u_i 和 p_j 的点击
        # 否则使用全局平均评分rating值(cold start 冷启动问题)
        if self.trainset.knows_user(i) and self.trainset.knows_item(j):
            TODO
            # 返回 u_i 和 p_j 的点击
        else:
            TODO
            # 返回 全局平均评分rating值

## 演示如何调用以上定义的矩阵分解类实现书籍商品的推荐

In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import os

file_path = os.path.expanduser('./book_crossing/book_ratings.dat')

TODO
# 创建 reader 对象

data = Dataset.load_from_file(file_path, reader=reader)

In [4]:
# 将数据随机分为训练和测试数据集
trainset, testset = train_test_split(data, test_size=.25)

TODO
# 初始化以上定义的矩阵分解类.

TODO
# 训练

TODO
# 预测

TODO
# 计算平均绝对误差

Fitting data with SGD...
MAE:  1.3518


1.3517903389238524

In [5]:
# 使用 surpise 内建的基于最近邻的方法做比较
algo = surprise.KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  1.4289


1.4288857409716478

In [6]:
# 使用 surpise 内建的基于 SVD 的方法做比较
algo = surprise.SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mae(predictions)

MAE:  1.1411


1.141127493999726